<a href="https://colab.research.google.com/github/MichaelHMorei/desafios/blob/main/Sicredi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://apache.osuosl.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop2.7.tgz

In [2]:
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

In [3]:
os.environ["SPARK_HOME"]

'/content/spark-3.2.1-bin-hadoop2.7'

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
pip install spark-nlp==2.4.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 108 kB 27.7 MB/s 


In [6]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.4.2
Apache Spark version:  3.2.1


In [79]:
#Importando bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark	
from pyspark.sql.types import *
from datetime import *

In [80]:
# Obtendo a data/hora atual
 
date1 = datetime.now()
date_atual = date1.strftime('%d/%m/%Y')
hora_Atual = date1.strftime('%H:%M:%S')

print('#### Processamento iniciado dia: '+date_atual+' as: '+hora_Atual+' ####' )

#### Processamento iniciado dia: 30/06/2022 as: 12:13:14 ####


In [81]:
#Lendo arquivos do google drive
from google.colab import drive
drive.mount("/content/gdrive")

# Lendo os arquivos e passando para uma variavel
df_Movimento = spark.read.csv("/content/gdrive/My Drive/Sicredi/Dim_Movimento.csv",inferSchema = 'True', sep =";", header ='True') 
df_Cartao = spark.read.csv("/content/gdrive/My Drive/Sicredi/Dim_Cartao.csv",inferSchema = 'True', sep =";", header ='True') 
df_Associado = spark.read.csv("/content/gdrive/My Drive/Sicredi/Dim_Associado.csv",inferSchema = 'True', sep =";", header ='True') 
df_Conta = spark.read.csv("/content/gdrive/My Drive/Sicredi/Dim_Conta.csv",inferSchema = 'True',sep =";" , header ='True') 


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [82]:
#Criando tabelas temporarias SQL

df_Movimento.createOrReplaceTempView('Temp_Movimento')
df_Cartao.createOrReplaceTempView('Temp_Cartao')
df_Associado.createOrReplaceTempView('Temp_Associado')
df_Conta.createOrReplaceTempView('Temp_Conta')

In [83]:
movimento_flat = spark.sql("""

    SELECT 
          CAST(nome AS STRING) AS nome_associado,
          CAST(sobrenome AS STRING) AS sobrenome_associado ,
          CAST(idade AS STRING) AS idade_associado,
          CAST( vlr_transacao AS STRING) AS vlr_transacao_movimento,
          CAST(des_transacao AS STRING) AS des_trasacao_movimento,
          CAST(data_movimento AS STRING) AS data_movimento,
          CAST(num_cartao AS STRING) AS numero_cartao,
          CAST(nome_impresso AS STRING) AS nome_impresso_cartao,
          CAST(tipo AS STRING) AS tipo_conta,
          CAST(data_criacao AS STRING) AS data_criacao_conta
         
  FROM Temp_Movimento 

    INNER JOIN Temp_Cartao ON
    Temp_Cartao.id = Temp_Movimento.id_cartao

    INNER JOIN Temp_Conta ON
    Temp_Conta.id = Temp_Cartao.id_conta

    INNER JOIN Temp_Associado ON
    Temp_Associado.id = Temp_Conta.id_associado

""")

In [85]:
movimento_flat.show(truncate=False)

+--------------+-------------------+---------------+-----------------------+----------------------+-----------------------+-------------+--------------------+--------------+-----------------------+
|nome_associado|sobrenome_associado|idade_associado|vlr_transacao_movimento|des_trasacao_movimento|data_movimento         |numero_cartao|nome_impresso_cartao|tipo_conta    |data_criacao_conta     |
+--------------+-------------------+---------------+-----------------------+----------------------+-----------------------+-------------+--------------------+--------------+-----------------------+
|Erick         |Moura              |51             |2200.0                 |Supemercado BH        |2022-06-29 21:03:13.130|13093354     |Erick Moura         |Poupanca      |2022-06-29 19:49:36.970|
|Noah          |Fogaça             |18             |1589.0                 |Mercado Livre         |2022-06-29 21:03:13.133|36036650     |Noah Fogaça         |Poupanca      |2022-06-29 19:49:36.970|
|Fernanda 

In [78]:
movimento_flat.write.format("csv").save("/content/gdrive/My Drive/Sicredi/movimento_flat.csv")

In [75]:
# Fim do processamento
#Horario de processamento

data_now = datetime.now()
data = data_now.strftime("%Y/%m/%d")
hora = data_now.strftime("%H:%M:%S")
print('### Processamento finalizado: '+data+' as '+hora+' ###')

### Processamento finalizado: 2022/06/30 as 11:59:48 ###
